In [1]:
import nltk.tokenize 
import pandas as pd
fullText = pd.read_csv('MainData_Turkish4.csv',encoding = 'utf-8',delimiter=',') #fullText
#print(fullText)
#all the value of our csv file in column 0 means our disease discription 
diseaseDescription = fullText.iloc[:,0]
#diseaseDescription
#all the value of our csv file in column 1 means our label 
#diseaseName = fullText.iloc[:,1]
#diseaseName

In [2]:
#CHANGE TO LOWERCASE
diseaseDescription = [item.lower() for item in diseaseDescription]


In [3]:
#remove white space from start and end of strings
diseaseDescription = [item.strip('') for item in diseaseDescription]
#diseaseDescription

In [4]:
#remove puncutations from diseaseDescription lists
import string
punc = string.punctuation
#print(punc)

diseaseDescriptionListAfterPunctuation = [''.join(char for char in eachItem if char not in string.punctuation) for eachItem in diseaseDescription]
#diseaseDescriptionListAfterPunctuation

In [5]:
#method two is the best for this case
#https://github.com/ahmetax/trstop/blob/master/dosyalar/turkce-stop-words
filee = open("turkishStopWords.txt", "r",encoding="utf-8")
turkishStopWords = filee.read()
turkishStopWords = turkishStopWords.split(',')
# print(turkishStopWords)
# for item in turkishStopWords: 
#     print(item) 

cleanedData_FromStopWords = [' '.join(w for w in word.split() if w not in turkishStopWords) for word in diseaseDescriptionListAfterPunctuation]
#print(cleanedData_FromStopWords)

listt = [item + ',|' for item in cleanedData_FromStopWords]
#listt

In [6]:
# first off all install zeyrek(for lemmitization) which is part of zembrek library
#pip install zeyrek into your env variable
import zeyrek
analyzer = zeyrek.MorphAnalyzer()
#print(analyzer.lemmatize('incelendi')[0][1][0])
Lemmatized_data = [analyzer.lemmatize(item) for item in listt]
#print(Lemmatized_data)

FinishedList = []
for item in Lemmatized_data:
    for member in item:
        FinishedList.append(member[1][0])
                
#print(FinishedList)

bad_char = [',']
resulttt = ' '.join(char for char in FinishedList if not char in bad_char) 
#resulttt
newlist =resulttt.split("|")
#print(newlist)
#remove last index which have just '' item

X = newlist[:-1]
#print(newlist)
#print(len(newlist))
#print(len(diseaseName))




In [7]:
# Load Data Viz Pkgs
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
# ML Pkgs
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss,classification_report
### Split Dataset into Train and Text
from sklearn.model_selection import LeaveOneOut,KFold
# Feature engineering
from sklearn.feature_extraction.text import TfidfVectorizer
# Multi Label Pkgs
# !pip install scikit-multilearn
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN



tfidf = TfidfVectorizer()
# Build Features
Xfeatures = tfidf.fit_transform(X).toarray()

labels = fullText[['POLİP','GASTRİT','HİATAL YETMEZLİK','ÜLSER','HİATAL HERNİ','DİĞER']]
#print(labels)

import skmultilearn
binary_rel_clf = BinaryRelevance(MultinomialNB())


acc = []
kf = KFold(n_splits=148)
for train_ids,test_ids in kf.split(Xfeatures,labels):
    train_docs,test_docs = Xfeatures[train_ids],Xfeatures[test_ids]
    train_y,test_y = labels.iloc[train_ids],labels.iloc[test_ids]
    clf = binary_rel_clf.fit(train_docs,train_y)
    pred_y = clf.predict(test_docs)
    acc.append(accuracy_score(test_y,pred_y))


In [8]:
total_accuracy = (sum(acc)/len(labels))*100
print(total_accuracy)

50.0
